In [ ]:
'''Pester/Dialogue/etc log scraper for homestuck.com'''
import csv
import requests
import re
from bs4 import BeautifulSoup

URL = 'https://www.homestuck.com/story/'

def parse_page(content):
    soup = BeautifulSoup(content)
    samples = soup.find_all('p', 'o_chat-log')
    if len(samples) < 1:
        return None
    messages = BeautifulSoup(f'<html>{samples[0]}</html>').find_all('span')
    if len(messages) < 1:
        return None
    
    output = []
    for message in messages:
        # This is a little lazy, but we're just going to drop any message with special formatting
        if message.string is None:
            continue
        stripped = message.string.strip()
        if ':' in stripped:
            color = re.search(r'.*(#\w{6})', message.attrs['style'])
            handle = re.search(r'(.*): ', stripped)
            message = re.search(r'.*: (.*)', stripped)
            if color is not None and handle is not None and message is not None:
                output.append((handle.group(1), color.group(1), message.group(1)))
    return output

for i in range(26, 8044): # Pages with logs: 26-8043
    result = requests.get(URL + str(i))
    if result.status_code == 200:
        page_messages = parse_page(result.content)
        if page_messages is not None and len(page_messages) > 0:
            with open(f'{i}.csv', mode='w', newline='') as target:
                fieldnames = ['handle', 'color', 'message']
                writer = csv.DictWriter(target, fieldnames=fieldnames)
                writer.writeheader()
                for row in page_messages:
                    writer.writerow({'handle': row[0], 
                                     'color': row[1], 
                                     'message': row[2]})
    else:
        print(f'Missed page due to failed request: {i}')

Missed page due to failed request: 2399
Missed page due to failed request: 3038
Missed page due to failed request: 3088
